# Muhammmed Yahya Avar -Y8OIJ1

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from lightgbm import early_stopping, log_evaluation
from xgboost import XGBRegressor
import pandas as pd
from sklearn.linear_model import Ridge

## Load Data

In [29]:
train = pd.read_csv("public_train.csv")
test = pd.read_csv("public_test.csv")
sample_sub = pd.read_csv("public_sample_submission.csv")

print("===TRAIN===")
display(train.head())

print("===TEST===")
display(test.head())

print("===SAMPLE SUBMISSION===")
display(sample_sub.head())

print("\n===TRAIN INFO===")
print(train.info())

print("\n===TEST INFO===")
print(test.info())

print("\n===SAMPLE SUBMISSION INFO===")
print(sample_sub.info())


===TRAIN===


,ID,deviceuniquecode,Trip_ID_first,Trip_ID_last,fuel_consumption_sum,duration_sum,speed_firstInTrip,speed_lastintrip,heading_diff_std,speed_mean,...,location_3,location_4,location_5,location_6,location_7,location_8,distance_1,distance_2,distance_3,distance_4
0,1,8,176849,176849.0,68.766667,40.666667,24.4444,23.8889,1.511858,24.012475,...,7.581883,2.047691,34.137916,3.028593,55.795315,2.133725,0.005250,0.886151,15863.69375,11044.80850
1,2,8,176850,176850.0,200.083333,42.458333,23.8889,23.8889,0.349927,23.888900,...,7.629009,2.046944,34.226847,3.028504,55.801277,2.133585,0.004649,0.884166,15864.45610,11045.33925
2,3,11,193394,NaN,217.291667,41.458333,23.8889,23.8889,0.599479,23.888900,...,0.366203,2.271124,14.292042,3.082449,69.077962,2.128446,0.000865,0.878730,19245.86545,13399.55100
3,4,14,215306,NaN,350.000000,40.000000,23.6111,23.3333,1.958758,23.441642,...,12.784224,1.934254,44.731700,2.995890,48.595126,2.191234,0.001445,0.880152,18467.86160,12857.85600
4,5,18,259243,259243.0,270.893333,41.908333,23.0556,23.6111,0.494872,23.434720,...,0.480065,2.262096,14.963864,3.078656,67.747127,2.133740,0.001291,0.899235,8697.54450,6055.71450


===TEST===


,ID,deviceuniquecode,Trip_ID_first,Trip_ID_last,duration_sum,speed_firstInTrip,speed_lastintrip,heading_diff_std,speed_mean,ambient_air_temperature_mean,...,location_3,location_4,location_5,location_6,location_7,location_8,distance_1,distance_2,distance_3,distance_4
0,180504,11,200261,200261.0,41.648276,23.0556,23.6111,1.178030,23.126397,65535.0,...,15.584240,1.880508,49.909732,2.981981,46.118184,2.215676,0.003467,0.882255,24609.84695,17133.89425
1,180505,14,214077,214077.0,40.200000,24.4444,24.7222,2.096440,24.624970,2.0,...,17.215691,1.853552,52.740443,2.976383,45.514553,2.224590,0.006274,0.885655,17517.98535,12196.59925
2,180506,21,5612,5612.0,44.882576,20.8333,24.4444,3.280244,22.036093,2.0,...,7.716784,2.421555,13.413842,3.192076,121.381587,1.667007,0.005264,0.888134,7960.54560,5542.59750
3,180507,5,14625,14625.0,42.291667,23.3333,23.3333,0.433013,23.365247,1.0,...,4.663400,2.110945,27.826891,3.043195,58.638776,2.122241,0.004488,0.895810,63246.04910,44032.51325
4,180508,23,29898,29898.0,46.004141,20.5556,22.2222,0.737028,21.430551,6.0,...,96.990387,-0.844042,169.958019,2.674481,26.050579,2.678368,0.001474,0.900456,15879.96380,11056.15500


===SAMPLE SUBMISSION===


,ID,fuel_consumption_sum
0,180504,0
1,180505,0
2,180506,0
3,180507,0
4,180508,0



===TRAIN INFO===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180503 entries, 0 to 180502
Data columns (total 62 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   ID                                           180503 non-null  int64  
 1   deviceuniquecode                             180503 non-null  int64  
 2   Trip_ID_first                                180503 non-null  int64  
 3   Trip_ID_last                                 169350 non-null  float64
 4   fuel_consumption_sum                         180503 non-null  float64
 5   duration_sum                                 180503 non-null  float64
 6   speed_firstInTrip                            180503 non-null  float64
 7   speed_lastintrip                             180503 non-null  float64
 8   heading_diff_std                             180503 non-null  float64
 9   speed_mean                               

## Feature Engineering Preprocess

In [30]:
def add_features(df):
    df = df.copy()
    # Speed patterns
    df['speed_change'] = df['speed_lastintrip'] - df['speed_firstInTrip']
    df['speed_stability'] = df['speed_mean'] / (df['heading_diff_std'] + 1)

    # Load & efficiency
    df['weight_speed_ratio'] = df['weight_1'] / (df['speed_mean'] + 1)
    df['engine_load_speed_ratio'] = df['engine_percent_load_at_current_speed_mean'] / (df['engine_speed_mean'] + 1)

    # Wind effects
    df['wind_impact'] = df['env_wind_kph'] * np.abs(np.cos(np.radians(df['env_wind_degrees_based_on_vehicle_direction'])))

    # Driving behavior
    df['aggressive_driving'] = df['accelerator_pedal_position_mean'] * df['brake_switch_mean']
    df['cruise_efficiency'] = df['cruise_control_active_mean'] * df['speed_mean']

    # Temperature effects
    df['temp_variance'] = np.abs(df['ambient_air_temperature_mean'] - df['engine_coolant_temperature_mean'])

    # Trip characteristics
    df['trip_intensity'] = df['duration_sum'] * df['speed_mean']

    # Interaction: heavy load on hills
    if 'road_level_approximation' in df.columns:
        df['load_terrain'] = df['weight_1'] * df['road_level_approximation'].fillna(0)

    return df

train_fe = add_features(train)
test_fe = add_features(test)


## TRI-MODEL ENSEMBLE CatBoost + LightGBM + XGBoost

In [31]:
TARGET = "fuel_consumption_sum"
ID_COL = "ID"

y = train_fe[TARGET].astype(float)
X = train_fe.drop(columns=[TARGET])
X_test = test_fe.copy()

test_ids = X_test[ID_COL].values

X = X.drop(columns=[ID_COL], errors="ignore")
X_test = X_test.drop(columns=[ID_COL], errors="ignore")

X_test = X_test.reindex(columns=X.columns, fill_value=np.nan)


# Train/val split for weights
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.10, random_state=42
)

# Define models
cat_params = dict(
    loss_function='MAE',
    iterations=50000,
    learning_rate=0.03,
    depth=7,
    l2_leaf_reg=3.0,
    bagging_temperature=0.8,
    random_strength=1.5,
    random_seed=42,
    verbose=500,
    eval_metric='MAE',
    early_stopping_rounds=300
)

lgb_params = dict(
    n_estimators=50000,
    learning_rate=0.03,
    num_leaves=63,
    colsample_bytree=0.8,
    subsample=0.8,
    reg_alpha=1.0,
    reg_lambda=2.0,
    random_state=42
)

xgb_params = dict(
    n_estimators=50000,
    max_depth=7,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    objective='reg:squarederror',
    random_state=42,
    tree_method="hist"
)

cat = CatBoostRegressor(**cat_params)
lgb = LGBMRegressor(**lgb_params)
xgb = XGBRegressor(**xgb_params)

# Fit on split (for MAE & best iters)
print("\n[LightGBM] Training...")
lgb.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="l1",
    callbacks=[
        early_stopping(stopping_rounds=300),
        log_evaluation(period=500)
    ]
)

best_lgb_iters = lgb.best_iteration_
lgb_val_pred = lgb.predict(X_val, num_iteration=best_lgb_iters)
mae_lgb = mean_absolute_error(y_val, lgb_val_pred)

print("\n[XGBoost] Training...")
xgb.fit(X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric="mae",
        verbose=300,
        early_stopping_rounds=700)
best_xgb_iters = getattr(xgb, "best_iteration", None)
xgb_val_pred = xgb.predict(X_val, iteration_range=(0, best_xgb_iters))
mae_xgb = mean_absolute_error(y_val, xgb_val_pred)

print("\n[CatBoost] Training...")
cat.fit(X_tr, y_tr, eval_set=(X_val, y_val), use_best_model=True)
cat_val_pred = cat.predict(X_val)
mae_cat = mean_absolute_error(y_val, cat_val_pred)
best_cat_iters = getattr(cat, "best_iteration_", None)


print("\n=== VALIDATION MAE ===")
print(f"CatBoost MAE:  {mae_cat:.6f}")
print(f"LightGBM MAE:  {mae_lgb:.6f}")
print(f"XGBoost  MAE:  {mae_xgb:.6f}")

# Compute weights (inverse MAE)
eps = 1e-9
w_cat = 1.0 / (mae_cat + eps)
w_lgb = 1.0 / (mae_lgb + eps)
w_xgb = 1.0 / (mae_xgb + eps)

w_sum = w_cat + w_lgb + w_xgb
w_cat /= w_sum
w_lgb /= w_sum
w_xgb /= w_sum

print("\n=== ENSEMBLE WEIGHTS (inverse-MAE normalized) ===")
print(f"CatBoost: {w_cat:.4f} | LightGBM: {w_lgb:.4f} | XGBoost: {w_xgb:.4f}")

# Retrain on FULL data using best iters

print("\n[LightGBM] Retraining on FULL data...")
lgb_full = LGBMRegressor(**lgb_params)
if best_lgb_iters is not None and best_lgb_iters > 0:
    lgb_full.set_params(n_estimators=best_lgb_iters)
lgb_full.fit(X, y)

print("\n[XGBoost] Retraining on FULL data...")
xgb_full = XGBRegressor(**xgb_params)
if best_xgb_iters is not None and best_xgb_iters > 0:
    xgb_full.set_params(n_estimators=best_xgb_iters)
xgb_full.fit(X, y)

print("\n[CatBoost] Retraining on FULL data...")
cat_full = CatBoostRegressor(**{**cat_params, "early_stopping_rounds": None})
# If best iteration exists, use it
if best_cat_iters is not None and best_cat_iters > 0:
    cat_full.set_params(iterations=best_cat_iters)
cat_full.fit(X, y)

# Predict test & blend
print("\nPredicting test with each model...")
pred_cat = cat_full.predict(X_test)
pred_lgb = lgb_full.predict(X_test)
pred_xgb = xgb_full.predict(X_test)

final_pred = w_cat * pred_cat + w_lgb * pred_lgb + w_xgb * pred_xgb

# Save submission
submission = pd.DataFrame({
    "ID": test_ids,
    "fuel_consumption_sum": final_pred
})
sub_name = "BME_EN_Y8OIJ1.csv"
submission.to_csv(sub_name, index=False)
print(f"\nSaved {sub_name}")


[LightGBM] Training...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12342
[LightGBM] [Info] Number of data points in the train set: 162452, number of used features: 69
[LightGBM] [Info] Start training from score 279.661600
Training until validation scores don't improve for 700 rounds
[500]	valid_0's l1: 19.4036	valid_0's l2: 904.472
[1000]	valid_0's l1: 19.0557	valid_0's l2: 872.623
[1500]	valid_0's l1: 18.8526	valid_0's l2: 855.272
[2000]	valid_0's l1: 18.7254	valid_0's l2: 845.274
[2500]	valid_0's l1: 18.6477	valid_0's l2: 839.276
[3000]	valid_0's l1: 18.5815	valid_0's l2: 835.196
[3500]	valid_0's l1: 18.5306	valid_0's l2: 831.791
[4000]	valid_0's l1: 18.5016	valid_0's l2: 829.983
[4500]	valid_0's l1: 18.4799	valid_0's l2: 828.394
[5000]	valid_0's l1: 18.4504	valid_0's l2: 826.53
[5500]	valid_0's l1: 18.4369	valid_0's l2: 824.888
[6000]	va

C:\Users\muhammedyahya.avar\AppData\Local\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\muhammedyahya.avar\AppData\Local\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[500]	validation_0-mae:19.07460
[1000]	validation_0-mae:18.70770
[1500]	validation_0-mae:18.54396
[2000]	validation_0-mae:18.44694
[2500]	validation_0-mae:18.39693
[3000]	validation_0-mae:18.35382
[3500]	validation_0-mae:18.31533
[4000]	validation_0-mae:18.29691
[4500]	validation_0-mae:18.29034
[5000]	validation_0-mae:18.27933
[5500]	validation_0-mae:18.27789
[6000]	validation_0-mae:18.27314
[6500]	validation_0-mae:18.27317
[6996]	validation_0-mae:18.27397

[CatBoost] Training...
0:	learn: 126.5152959	test: 125.6163306	best: 125.6163306 (0)	total: 75.2ms	remaining: 1h 2m 41s
500:	learn: 20.6913616	test: 20.8388170	best: 20.8388170 (500)	total: 32.4s	remaining: 53m 20s
1000:	learn: 19.3996320	test: 19.8127300	best: 19.8127300 (1000)	total: 1m 4s	remaining: 52m 59s
1500:	learn: 18.7836670	test: 19.4467824	best: 19.4467824 (1500)	total: 1m 37s	remaining: 52m 39s
2000:	learn: 18.3319662	test: 19.2472382	best: 19.2472382 (2000)	total: 2m 10s	remaining: 52m 6s
2500:	learn: 17.9808444	test: 1

## Submission Check

In [33]:
import pandas as pd

sub = pd.read_csv("BME_EN_Y8OIJ1.csv")   

print("=== BASIC INFO ===")
print(sub.info())
print()

print("Rows:", len(sub))
if len(sub) == 189469:
    print("✓ Correct number of rows")
else:
    print("✗ Incorrect number of rows! Expected 189469")

print()
required_cols = ["ID", "fuel_consumption_sum"]
print("=== Checking columns ===")
print("Columns found:", list(sub.columns))

if list(sub.columns) == required_cols:
    print("✓ Columns match exactly")
else:
    print("✗ Column mismatch! Expected:", required_cols)

print()

test = pd.read_csv("public_test.csv")

if set(sub["ID"]) == set(test["ID"]):
    print("✓ ID values match test file")
else:
    print("✗ ID mismatch with test file!")

# 4. Check for NaN predictions
if sub["fuel_consumption_sum"].isna().sum() == 0:
    print("✓ No missing predictions")
else:
    print("✗ Missing values found in predictions!")

print("\n=== DONE ===")


=== BASIC INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189469 entries, 0 to 189468
Data columns (total 2 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    189469 non-null  int64  
 1   fuel_consumption_sum  189469 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 2.9 MB
None

Rows: 189469
✓ Correct number of rows

=== Checking columns ===
Columns found: ['ID', 'fuel_consumption_sum']
✓ Columns match exactly

✓ ID values match test file
✓ No missing predictions

=== DONE ===
